# Wykonaj następujące polecenia:

## Machine Learning intro [60%]
1. załaduj dane z pliku `train.csv` do dataframe [10%]
2. analogicznie do ćwiczeń z zajęć stwórz nowe zmienne wykonując następujące transformacje:
    - `MSZoning` - one hot encoding [10%]
    - `LotShape` - one hot encoding [10%]
    - `LotArea` - skalowanie od 0 do 1 [10%]
    - `OverallCond` - standaryzacja [10%]
3. Utworzone nowe zmienne dołącz do istniejącego zbioru (dataframe'u z pkt 1), w nazwie nowych kolumn po one hot encoding umieść kategorię której odpowiada:     
Przykładowo dla zmiennej `LotShape` powstaną 4 nowe kolumny o nazwach: `'LotShape_IR1'`, `'LotShape_IR2'`, `'LotShape_IR3'`, `'LotShape_Reg'`. [10%]

## Regresja logistyczna [40%]

Klasyfikator dla zbioru breast cancer
Zbuduj klasyfikator modelem LogisticRegression

Dane znajdują się tutaj:
from sklearn.datasets import load_breast_cancer

1. Wczytaj data jako X i target jako y [10%]
2. Przeanalizuj kształt danych [10%]
3. Podziel dane na zbiór treningowy i testowy metodą train_test_split
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html [10%]
4. Dopasuj model na części treningowej i policz score dopasowania do danych [10%]

In [5]:
%matplotlib inline
from IPython.core.display import display, HTML
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from dtreeviz.trees import *
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from mlxtend import plotting
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn import datasets
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

## Machine learning intro

In [6]:
df = pd.read_csv("train.csv")


One hot Encoding

In [7]:
one_hot1=df[['MSZoning','LotShape']]

my_encoder = OneHotEncoder(sparse=False)
my_encoder.fit(one_hot1)

one_hot1_ohe=my_encoder.transform(one_hot1)
one_hot=pd.DataFrame(one_hot1_ohe)

new_names=[]
for i in my_encoder.categories_[0]:
        new_names.append('MSZoning_{}'.format(i))
for i in my_encoder.categories_[1]:
        new_names.append('LotShape_{}'.format(i))

one_hot=one_hot.rename(columns=dict(zip(one_hot.columns,new_names)))
one_hot.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


Skalowanie 0-1

In [8]:
scal=df[['LotArea']]
my_scaler = MinMaxScaler()
my_scaler.fit(scal)
scal_out=my_scaler.transform(scal)
scal_out=pd.DataFrame(scal_out)
scal_out=scal_out.rename(columns={0:'LotArea_scal'})
scal_out.head()

,LotArea_scal
0,0.033420
1,0.038795
2,0.046507
3,0.038561
4,0.060576


standaryzacja

In [9]:
stand=df[['OverallCond']]
my_stand = StandardScaler()
my_stand.fit(stand)
stand_out = my_stand.transform(stand)
stand_out=pd.DataFrame(stand_out)
stand_out=stand_out.rename(columns={0:'OverallCond_stand'})
stand_out.head()

,OverallCond_stand
0,-0.517200
1,2.179628
2,-0.517200
3,-0.517200
4,-0.517200


In [10]:
df=pd.concat([df,one_hot,scal_out, stand_out])
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

## Regresja logistyczna

In [11]:
breast=load_breast_cancer()
X,y=breast.data, breast.target


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 101)

In [13]:
model_lin = LogisticRegression().fit(X_train,y_train)

pred_lin_train =  model_lin.predict(X_train)
pred_lin_test =  model_lin.predict(X_test)

print('Regresja logistyczna trening accuracy:',accuracy_score(pred_lin_train,y_train))
print('Regresja logistyczna test accuracy:',accuracy_score(pred_lin_test,y_test))

Regresja logistyczna trening accuracy: 0.9547738693467337
Regresja logistyczna test accuracy: 0.9298245614035088
/home/gosia/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
